## EDA / data cleanup part 2
Attempt to add categories for food outlets

In [2]:
# import libraries
import os
import sys

import pandas as pd
import numpy as np
import scipy as sp

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import random
import time

from PIL import Image

import wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import ast
import re


In [3]:
df_ig = pd.read_csv(r'data\df_ig.csv', index_col=0)

In [4]:
df_ig.shape

(15210, 56)

In [8]:
df_ig.head(3)

,file_name,descrip,hashtags,comments_disabled,__typename,id,edge_media_to_caption,shortcode,edge_media_to_comment,taken_at_timestamp,...,is_published,title,video_duration,edge_sidecar_to_children,username,location_name,location_desc,Unnamed: 0.1,Unnamed: 0.1.1,timestamp
0,2020-03-09_15-23-42_UTC,Sorrel OC by @slavontherocks done by @neptunea...,"['goku', 'vegeta', 'gohan', 'trunks', 'cabba',...",False,GraphImage,2.261169e+18,{'edges': [{'node': {'text': 'Planted the #kal...,B9hSO10JAC8,{'count': 0},1.583772e+09,...,NaN,NaN,NaN,NaN,derbyka,NaN,NaN,NaN,NaN,2020-03-09 16:45:32
1,2020-03-09_15-24-07_UTC,Green juice going out this morning to my sis @...,"['NettesNaturalJuice', 'GreenJuice', 'Kale', '...",False,GraphImage,2.261168e+18,"{'edges': [{'node': {'text': ""This post has be...",B9hSHwVgiG6,{'count': 0},1.583772e+09,...,NaN,NaN,NaN,NaN,saltsugarspice,NaN,NaN,NaN,NaN,2020-03-09 16:44:34
2,2020-03-09_15-24-51_UTC,Herkesin memleketi cennettir kendine. #kale #t...,"['kale', 'tillo', 'mavi', 'kırmızı', 'doğa', '...",False,GraphImage,2.261167e+18,{'edges': [{'node': {'text': 'Eşsiz Lezzetler ...,B9hR3yqFmPX,{'count': 0},1.583772e+09,...,NaN,NaN,NaN,NaN,akropol_bistro,Siirt Kale,Siirt Kale\nhttps://maps.google.com/maps?q=37....,NaN,NaN,2020-03-09 16:42:23


In [6]:
# df_ig.columns

### Missing values

In [7]:
# # count missing values
# df_ig.isnull().sum()

In [9]:
# remove missing file names
df_ig = df_ig.dropna(subset=['file_name'])

In [84]:
# df_ig2 = df_ig[["file_name", "timestamp", "hashtags", "descrip", "accessibility_caption", "username", "owner", "display_url", "id", "shortcode", "is_ad", "has_ranked_comments", "location", "location_name", "location_desc" ]]
df_ig2 = df_ig[["file_name", "timestamp", "hashtags", "descrip", "accessibility_caption", "username", "owner", "display_url", "id", "shortcode", "is_ad", "has_ranked_comments", "location"]]

In [85]:
print("Total shape:", df_ig2.shape)
print(df_ig2.isnull().sum())

Total shape: (15206, 13)
file_name                    0
timestamp                 4846
hashtags                     4
descrip                      4
accessibility_caption     5598
username                 10234
owner                     4846
display_url               4846
id                        4846
shortcode                 4846
is_ad                     4846
has_ranked_comments       4846
location                 12210
dtype: int64


In [86]:
# Running count of loc data
print(f"Of the {df_ig2.shape[0]} posts, {df_ig2['location'].notnull().sum()} have location data.")
# print(f"Of the {df_ig2.shape[0]} posts, {df_ig2['location_name'].notnull().sum()} have location_name data.")

# print(f"Of the {df_ig2.shape[0]} posts, {df_ig2['location'].isnull().sum()} do not have location data.")


Of the 15206 posts, 2996 have location data.


In [88]:
# # how many have both location and/or location_name data? do they intersect?

withloc_df = df_ig2[df_ig2['location'].notnull()]
# withlocname_df = df_ig2[df_ig2['location_name'].notnull()]

# withloc_df['all_loc_info'] = withloc_df['location']
# withlocname_df['all_loc_info'] = withlocname_df['location_name']
# bothloc_df = pd.merge(withloc_df, withlocname_df, how='inner', on=['all_loc_info'])
# print(bothloc_df.shape)
# # results show NO common rows have both location and location_name data

In [89]:
# convert location field to dict
# IF the location_name field is NaN, populate with 'name' value from location dict

withloc_df['location'] = withloc_df['location'].apply(lambda x: ast.literal_eval(x) if x is not None else None)

In [90]:
# get the value assoc with 'name'

def getname(d):
    return d["name"]

withloc_df['loc_name_2'] = withloc_df['location'].apply(getname)
withloc_df['loc_name_2'] = withloc_df['loc_name_2'].str.strip()

In [94]:
# withloc_df.sample(10)

In [107]:
location_names = withloc_df['loc_name_2']
location_names.to_csv('location_names.csv', index=None)

In [11]:
## add an outlet category row for those posts with locations
categories = pd.read_csv(r"data\2014_outlet_categories.csv", index_col=0)
categories.rename(columns={"shop_name": "location_name", "type": "category"}, inplace=True)
categories['category']= categories['category'].str.replace("Burgers", "Fast Food")
categories.drop_duplicates(inplace=True)
categories.dropna(inplace=True)
print(categories.shape)

(4962, 2)


In [12]:
categories['category'].unique()

array(['Fast Food', 'French', 'Desserts', 'Sandwiches', 'Ice Cream',
       'Street Food', 'Bakery', 'Fried Chicken', 'Tacos', 'Caribbean',
       'Latin American', 'Mexican', 'Japanese', 'Pizza', 'South American',
       'American', 'BBQ', 'Seafood', 'Asian', 'Coffee Shop', 'Chinese',
       'Mediterranean', 'Indian', 'Korean', 'Juice Bar', 'Food', 'Donuts',
       'Italian', 'Diner', 'Greek', 'Sushi', 'Hot Dogs', 'Arepas',
       'Brewery', 'Bagels', 'Burritos', 'Snacks', 'Cupcakes', 'Wings',
       'New American', 'Eastern European', 'Thai', 'Dim Sum', 'Spanish',
       'Cuban', 'Argentinian', 'Steakhouse', 'Brazilian', 'Ethiopian',
       'Tea Room', 'Gastropub', 'African', 'Australian', 'Middle Eastern',
       'Peruvian', 'Salad', 'Vietnamese', 'Tapas', 'Moroccan', 'Winery',
       'Gluten-free', 'Dumplings', 'Swiss', 'Falafel', 'German', 'Soup',
       'Paella', 'Malaysian', 'Filipino', 'Indonesian',
       'Molecular Gastronomy', 'Mac & Cheese'], dtype=object)

In [13]:
withlocname = df_ig2[df_ig2['location_name'].notnull()]

# how many unique food outlets?
print(withlocname['location_name'].nunique())

2770


In [14]:
# naive join (without any fuzzy matching)
match1 = withlocname.merge(categories, on=['location_name'])
print(match1.shape)


(45, 16)


In [16]:
match1.sample(5)

,file_name,timestamp,hashtags,descrip,accessibility_caption,username,owner,display_url,id,shortcode,is_ad,has_ranked_comments,location,location_name,location_desc,category
26,2020-02-23_04-47-01_UTC,NaN,"['burgerking', 'bk', 'whopper', 'burger', 'bur...",Late night Bacon & Cheese Whopper & Fries.\n🍔\...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Burger King,Burger King\nhttps://maps.google.com/maps?q=36...,Fast Food
11,2020-02-18_19-52-48_UTC,2020-02-21 19:15:24,[],Animal style 🤪\n,Image may contain: food,pound.mx,"{'id': '7580812187', 'is_verified': False, 'pr...",https://scontent-mia3-2.cdninstagram.com/v/t51...,2.248923e+18,B81x3J_He64,False,False,NaN,In-N-Out Burger,In-N-Out Burger\nhttps://maps.google.com/maps?...,Fast Food
16,2020-02-23_04-18-50_UTC,NaN,"['In', 'inandoutburger', 'burger', 'keizeroreg...",Just chillin. 😎\n.\n.\n.\n.\n.\n.\n.\n.\n#In-N...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In-N-Out Burger,In-N-Out Burger\nhttps://maps.google.com/maps?...,Fast Food
23,2020-03-10_21-23-26_UTC,2020-03-10 20:49:05,"['burguerking', 'hamburguesas', 'hamburguer', ...","Burguer King me paga millones, olvidate.\n.\nP...",Image may contain: food,drayuheats,"{'id': '2122526598', 'is_verified': False, 'pr...",https://scontent-mia3-2.cdninstagram.com/v/t51...,2.262016e+18,B9kS5hQH3PK,False,False,"{'id': 673643029416527, 'name': 'Krispy Krunch...",Burger King,Burger King\nhttps://maps.google.com/maps?q=-3...,Fast Food
39,2020-02-22_04-00-00_UTC,NaN,[],Back from the groomers and looking like fluffe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hachi,Hachi\nhttps://maps.google.com/maps?q=-37.8399...,Sushi


In [23]:
unique_names = pd.Series(withlocname['location_name'].unique())
unique_names = unique_names.sort_values(ascending=True)
print(unique_names.shape)
unique_names.to_csv('unique_names.csv', index=False )

(2770,)

#### TODO: 
- Clean the unique names (remove double quotes; standardize/combine names using fuzzywuzzy or pyenchant)
- 